In [1]:
#Instalamos las librerias
#!pip install yfinance
#!pip install backtesting
#!pip install TA-Lib

In [1]:
#Importamos las librerias
from backtesting import Backtest, Strategy
from backtesting.lib import crossover,plot_heatmaps

import numpy
import talib
import pandas as pd

import tpqoa

from datetime import datetime, timedelta
import time

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
api = tpqoa.tpqoa("oanda.cfg")

# Backtesting

### Tiempo: 100 días
### Granularidad: H1

In [6]:
days = 100
now = datetime.utcnow()
now = now - timedelta(microseconds = now.microsecond) - timedelta(days = 6)
past = now - timedelta(days = days)

/var/folders/3v/201_zlm54219lq129m1qtzpc0000gn/T/ipykernel_5203/1181886766.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


In [8]:
df = api.get_history(instrument = "EUR_USD", start=past, end=now,
                granularity = "H1", price = "M", localize = False)

df

,o,h,l,c,volume,complete
time,,,,,,
2024-05-21 15:00:00+00:00,1.08592,1.08613,1.08530,1.08541,2728,True
2024-05-21 16:00:00+00:00,1.08540,1.08568,1.08522,1.08540,2070,True
2024-05-21 17:00:00+00:00,1.08538,1.08544,1.08464,1.08533,1813,True
2024-05-21 18:00:00+00:00,1.08535,1.08556,1.08502,1.08521,1329,True
2024-05-21 19:00:00+00:00,1.08522,1.08563,1.08509,1.08558,1251,True
...,...,...,...,...,...,...
2024-08-29 11:00:00+00:00,1.10975,1.10998,1.10914,1.10962,3309,True
2024-08-29 12:00:00+00:00,1.10962,1.10984,1.10728,1.10842,10412,True
2024-08-29 13:00:00+00:00,1.10842,1.10896,1.10727,1.10822,8696,True


In [9]:
df = df.rename(columns={'o':'Open', 'h':'High', 'l': 'Low', 'c':'Close', 'volume': 'Volume'})
df.drop('complete', axis=1, inplace=True)
df

,Open,High,Low,Close,Volume
time,,,,,
2024-05-21 15:00:00+00:00,1.08592,1.08613,1.08530,1.08541,2728
2024-05-21 16:00:00+00:00,1.08540,1.08568,1.08522,1.08540,2070
2024-05-21 17:00:00+00:00,1.08538,1.08544,1.08464,1.08533,1813
2024-05-21 18:00:00+00:00,1.08535,1.08556,1.08502,1.08521,1329
2024-05-21 19:00:00+00:00,1.08522,1.08563,1.08509,1.08558,1251
...,...,...,...,...,...
2024-08-29 11:00:00+00:00,1.10975,1.10998,1.10914,1.10962,3309
2024-08-29 12:00:00+00:00,1.10962,1.10984,1.10728,1.10842,10412
2024-08-29 13:00:00+00:00,1.10842,1.10896,1.10727,1.10822,8696


### Estrategia

In [11]:
class SMAcross(Strategy):

  slPercent = 0.95
  smaquickp = 50
  smaslowp = 200

  def init(self):
    self.smaQuick = self.I(talib.SMA,self.data.Close,self.smaquickp)
    self.smaSlow = self.I(talib.SMA,self.data.Close,self.smaslowp)

  def next(self):

    price = self.data.Close[-1]

    if crossover(self.smaQuick,self.smaSlow):
      self.buy(sl = price * self.slPercent)
    elif crossover(self.smaSlow,self.smaQuick):
      self.sell(sl = price + price * self.slPercent)


## Backtesting sin comisión

In [11]:
btbrok_sc = Backtest(df,SMAcross,cash=10000, exclusive_orders=True)#Ejecutamos el backtest
statsbrok_sc = btbrok_sc.run()
statsbrok_sc

Start                     2024-05-12 21:00...
End                       2024-08-20 18:00...
Duration                     99 days 21:00:00
Exposure Time [%]                   88.157895
Equity Final [$]                   9944.88373
Equity Peak [$]                   10041.92825
Return [%]                          -0.551163
Buy & Hold Return [%]                3.230718
Return (Ann.) [%]                   -1.588141
Volatility (Ann.) [%]                4.896865
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.767289
Avg. Drawdown [%]                   -1.267115
Max. Drawdown Duration       89 days 00:00:00
Avg. Drawdown Duration       29 days 18:00:00
# Trades                                   10
Win Rate [%]                             30.0
Best Trade [%]                       1.679381
Worst Trade [%]                     -0.875903
Avg. Trade [%]                    

## Backtesting con comisión

In [15]:
btbrok = Backtest(df,SMAcross,cash=10000, commission=.0015, exclusive_orders=True)

In [17]:
#Ejecutamos el backtest
statsbrok = btbrok.run()

#Vamos los resultados del backtest
statsbrok

Start                     2024-05-21 15:00...
End                       2024-08-29 15:00...
Duration                    100 days 00:00:00
Exposure Time [%]                   87.735311
Equity Final [$]                  9917.406957
Equity Peak [$]                  10054.858348
Return [%]                           -0.82593
Buy & Hold Return [%]                2.127307
Return (Ann.) [%]                   -2.373659
Volatility (Ann.) [%]                5.158126
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.465978
Avg. Drawdown [%]                   -0.901359
Max. Drawdown Duration       86 days 15:00:00
Avg. Drawdown Duration       21 days 18:00:00
# Trades                                    7
Win Rate [%]                        42.857143
Best Trade [%]                       1.105134
Worst Trade [%]                     -1.027444
Avg. Trade [%]                    

In [19]:
btbrok.plot()

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1325', ...)

## Optimización del SL y SMA

In [21]:
stats = btbrok.optimize(

    slPercent = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99],
    smaquickp = [10,20,30,40,50,60,70,80], 
    smaslowp = [150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250],
    maximize = "Return [%]",
)

stats

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 880 configurations.
  output = _optimize_grid()
/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/13 [00:00<?, ?it/s]

Start                     2024-05-21 15:00...
End                       2024-08-29 15:00...
Duration                    100 days 00:00:00
Exposure Time [%]                   81.460354
Equity Final [$]                 10254.726339
Equity Peak [$]                  10336.096593
Return [%]                           2.547263
Buy & Hold Return [%]                2.127307
Return (Ann.) [%]                    7.557859
Volatility (Ann.) [%]                4.861962
Sharpe Ratio                         1.554487
Sortino Ratio                        2.428865
Calmar Ratio                         5.226654
Max. Drawdown [%]                   -1.446023
Avg. Drawdown [%]                   -0.329003
Max. Drawdown Duration       26 days 23:00:00
Avg. Drawdown Duration        2 days 23:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                       1.586018
Worst Trade [%]                     -0.302888
Avg. Trade [%]                    

In [22]:
stats["_strategy"]

<Strategy SMAcross(slPercent=0.9,smaquickp=30,smaslowp=220)>

## Cálculo del backtesting con los nuevos parámetros

In [25]:
class SMAcross(Strategy):

  slPercent = 0.9
  smaquickp = 30
  smaslowp = 220

  def init(self):
    self.smaQuick = self.I(talib.SMA,self.data.Close,self.smaquickp)
    self.smaSlow = self.I(talib.SMA,self.data.Close,self.smaslowp)

  def next(self):

    price = self.data.Close[-1]

    if crossover(self.smaQuick,self.smaSlow):
      self.buy(sl = price * self.slPercent)
    elif crossover(self.smaSlow,self.smaQuick):
      self.sell(sl = price + price * self.slPercent)

In [27]:
btbrok_opt = Backtest(df,SMAcross,cash=10000, commission=.0015, exclusive_orders=True)#Ejecutamos el backtest
statsbrok_opt = btbrok_opt.run()
statsbrok_opt

Start                     2024-05-21 15:00...
End                       2024-08-29 15:00...
Duration                    100 days 00:00:00
Exposure Time [%]                   81.460354
Equity Final [$]                 10254.726339
Equity Peak [$]                  10336.096593
Return [%]                           2.547263
Buy & Hold Return [%]                2.127307
Return (Ann.) [%]                    7.557859
Volatility (Ann.) [%]                4.861962
Sharpe Ratio                         1.554487
Sortino Ratio                        2.428865
Calmar Ratio                         5.226654
Max. Drawdown [%]                   -1.446023
Avg. Drawdown [%]                   -0.329003
Max. Drawdown Duration       26 days 23:00:00
Avg. Drawdown Duration        2 days 23:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                       1.586018
Worst Trade [%]                     -0.302888
Avg. Trade [%]                    

In [29]:
btbrok.plot()

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1681', ...)